|  name             | email            | github      |
| ------            | ------           | ------      |
|Patrick Vongphakdy | pvong2@uic.edu   | patvong224  |
|Brian Zbojna       | bzbojn2@uic.edu  | bzbojn2     |
|Michael Carnowell  | mcarno2@uic.edu  | carnowell   |

https://github.com/CS418/group-project-team-import-antigravity

# Project Introduction


## Problem Statement and Questions

## Data

# DNF Analysis

## Data Cleaning

## Exploratory Data Analysis

## Machine Learning and Statistics

## Results

# Pit Stop Analysis

## Data Cleaning

## Exploratory Data Analysis

## Machine Learning and Statistics

## Results

# Competition Analysis

## Data Cleaning

## Exploratory Data Analysis

## Machine Learning and Statistics

## Results

# References